<a href="https://colab.research.google.com/github/vignesh374/vignesh03/blob/main/Traffic_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [2]:
!pip install scikit-learn
!pip install openTSNE
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

from sklearn import manifold
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from umap import UMAP

In [4]:
df = pd.read_csv("/content/Traffic.csv")

In [5]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekends = ['Saturday', 'Sunday']

df['Day Type'] = df['Day of the week'].apply(lambda x: 'Weekend' if x in weekends else 'Weekday')
df

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation,Day Type
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low,Weekday
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low,Weekday
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low,Weekday
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low,Weekday
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal,Weekday
...,...,...,...,...,...,...,...,...,...,...
2971,10:45:00 PM,9,Thursday,16,3,1,36,56,normal,Weekday
2972,11:00:00 PM,9,Thursday,11,0,1,30,42,normal,Weekday
2973,11:15:00 PM,9,Thursday,15,4,1,25,45,normal,Weekday
2974,11:30:00 PM,9,Thursday,16,5,0,27,48,normal,Weekday


In [6]:
meta_cols = ["Time", "Date", "Day of the week", "Traffic Situation", "Day Type"]
meta_df = df[meta_cols]
proj_df = df.drop(columns=meta_cols)

In [7]:
proj_df

,CarCount,BikeCount,BusCount,TruckCount,Total
0,31,0,4,4,39
1,49,0,3,3,55
2,46,0,3,6,55
3,51,0,2,5,58
4,57,6,15,16,94
...,...,...,...,...,...
2971,16,3,1,36,56
2972,11,0,1,30,42
2973,15,4,1,25,45
2974,16,5,0,27,48


In [8]:
meta_df

,Time,Date,Day of the week,Traffic Situation,Day Type
0,12:00:00 AM,10,Tuesday,low,Weekday
1,12:15:00 AM,10,Tuesday,low,Weekday
2,12:30:00 AM,10,Tuesday,low,Weekday
3,12:45:00 AM,10,Tuesday,low,Weekday
4,1:00:00 AM,10,Tuesday,normal,Weekday
...,...,...,...,...,...
2971,10:45:00 PM,9,Thursday,normal,Weekday
2972,11:00:00 PM,9,Thursday,normal,Weekday
2973,11:15:00 PM,9,Thursday,normal,Weekday
2974,11:30:00 PM,9,Thursday,normal,Weekday


In [9]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(proj_df)

tsne = TSNE(n_components=2, perplexity=50)
tsne_results = tsne.fit_transform(scaled_features)

umap = UMAP(n_components=2)
umap_results = umap.fit_transform(scaled_features)

pca = PCA(n_components=2)
pca_results = pca.fit_transform(scaled_features)

In [10]:
print(tsne_results.shape)
print(umap_results.shape)
print(pca_results.shape)

(2976, 2)
(2976, 2)
(2976, 2)


In [11]:
# turn coords into pandas dataframe
df_tsne_coords = pd.DataFrame(tsne_results, columns=['X','Y'])
df_umap_coords = pd.DataFrame(umap_results, columns=['X','Y'])
df_pca_coords = pd.DataFrame(pca_results, columns=['X','Y'])

In [12]:
alt.Chart(df_umap_coords).mark_circle(
    opacity=0.6
).encode(
    x='X',
    y='Y',
).properties(
    width=500,
    height=400,
    title="Projected Traffic Data"
).interactive()

alt.Chart(...)

In [13]:
combined_traffic = pd.concat([meta_df, df_umap_coords], axis='columns')

alt.Chart(combined_traffic).mark_point(
    opacity=0.6
).encode(
    x='X',
    y='Y',
    color='Traffic Situation:N',
).properties(
    width=700,
    height=700,
    title="Projected States"
).interactive()

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

alt.Chart(...)

In [14]:
combined_traffic = combined_traffic.rename_axis('index1').reset_index()

In [15]:
alt.Chart(combined_traffic).mark_line(
    opacity=0.6
).encode(
    x='X',
    y='Y',
    color='Traffic Situation:N', #color lines by solving attempt
    order='index:Q' # connect them in order (instead of position on x-axis)
).properties(
    width=700,
    height=700,
    title="Paths of solving attempts"
).interactive()

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

alt.Chart(...)